# Dask Workflow - Local

Demo notebooks 02a and 02b will show you the recommended way to work with Dask on a cluster.

Requirements:
 - Cluster has been added and configured, as shown in notebook 01.
 - `idact` is installed on the cluster using pip.

## Initial setup

Add `idact` to path and import it:

In [1]:
import sys
import os
import bitmath
import logging
import subprocess

sys.path.append('../')  # For running in repo. Unnecessary with pip install.
from idact import *

## Load cluster

In [2]:
load_environment()
cluster = show_cluster("pro")  # replace with your cluster name if necessary
cluster

Cluster(pro.cyfronet.pl, 22, plggarstka, auth=AuthMethod.PUBLIC_KEY, key='C:\\Users\\Maciej/.ssh\\id_rsa_j7', install_key=False, disable_sshd=False)

Make sure it's properly configured. Replace the following with correct values for your cluster, or skip if already configured:

In [3]:
cluster.config.setup_actions.jupyter = ['module load plgrid/tools/python-intel/3.6.2']
cluster.config.setup_actions.dask = cluster.config.setup_actions.jupyter
cluster.config.scratch = '$SCRATCH'

save_environment()

Get access node:

In [4]:
node = cluster.get_access_node()
node

Node(pro.cyfronet.pl:22, None)

Make sure authentication is set up correctly:

In [5]:
node.connect()
save_environment()

In [6]:
node.run('whoami')

'plggarstka'

In [7]:
node.run('hostname')

'login01.pro.cyfronet.pl'

## Allocate nodes

You will need nodes to deploy Jupyter Notebook and Dask on:

In [8]:
nodes = cluster.allocate_nodes(nodes=2,
                               cores=2,
                               memory_per_node=bitmath.GiB(10),
                               walltime=Walltime(minutes=20),
                               native_args={
                                   '--partition': 'plgrid-testing',
                                   '--account': 'intdata'
                               })

2018-11-15 00:29:28 INFO: Creating the ssh directory.


In [9]:
nodes

Nodes([Node(NotAllocated),Node(NotAllocated)], SlurmAllocation(job_id=14194946))

Wait until the nodes are allocated:

In [10]:
nodes.wait()
nodes

Nodes([Node(p0097:57204, 2018-11-14 23:49:33.998070+00:00),Node(p0100:44128, 2018-11-14 23:49:33.998070+00:00)], SlurmAllocation(job_id=14194946))

## Deploy notebook

You will work from a remote Jupyter Notebook deployed on the cluster:

In [11]:
nb = nodes[0].deploy_notebook()
nb

JupyterDeployment(8080 -> Node(p0097:57204, 2018-11-14 23:49:33.998070+00:00)

Open the remote notebook in a new tab:

In [12]:
nb.open_in_browser()

## Push nodes

To deploy Dask on the notebook, you will need access to nodes you allocated earlier.

Push the nodes to the cluster. You will be able to access them from there by calling `pull_deployments`.

In [13]:
cluster.push_deployment(nodes)

2018-11-15 00:30:18 INFO: Pushing deployment: Nodes([Node(p0097:57204, 2018-11-14 23:49:33.998070+00:00),Node(p0100:44128, 2018-11-14 23:49:33.998070+00:00)], SlurmAllocation(job_id=14194946))


## Push environment

If this is your first time working on the cluster from a remote notebook,
you may want to push the current environment. Alternatively, just `add_cluster` and perform other configuration steps on the remote notebook, as demonstrated in demo notebook 01.

In [14]:
push_environment(cluster)

2018-11-15 00:30:23 INFO: Pushing the environment to cluster.


## Copy next notebook to the cluster

Drag and drop `02b-DaskWorkflow-Remote.ipynb` to the deployed notebook, and open it there.

## Follow the instructions in notebook 02b

Follow instructions until you are referred back to this notebook.

## View Dask diagnostics

Dask deployment synchronization is currently not implemented.

To view Dask scheduler dashboard, you will need to open the tunnel manually.
Copy the port number from client description displayed on the remote notebook.

In [15]:
port = 55995  # copy port from other notebook

In [16]:
tunnel = nodes[0].tunnel(here=port, there=port)

Open the dashboard by executing the following, or just click the client description, as it should work now.

In [17]:
import webbrowser
webbrowser.open("http://localhost:{here}/status".format(here=tunnel.here));

In [18]:
tunnel.close()

## Monitor node resources

While working with Dask, it may be useful to monitor resource usage on nodes:

In [19]:
nodes[0].resources.memory_total

GiB(10.0)

In [20]:
nodes[0].resources.cpu_cores

2

In [21]:
nodes[0].resources.memory_usage

GiB(0.43328857421875)

In [22]:
nodes[1].resources.memory_usage

GiB(0.2129669189453125)

In [23]:
nodes[0].resources.cpu_usage

9.0

In [24]:
nodes[1].resources.cpu_usage

7.0

## Cancel Dask deployment

Cancel Dask deployment in the remote notebook, or just close it.

## Cancel other deployments

If the nodes are still running, make sure you cancel their allocation to save CPU time.

In [25]:
nodes.running()

True

In [26]:
nb.cancel()

2018-11-15 00:34:02 INFO: Cancelling Jupyter deployment.


In [27]:
nodes.cancel()

2018-11-15 00:34:08 INFO: Cancelling job 14194946.


In [28]:
nodes.running()

False

In [29]:
node.run('squeue')

'JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)\n          14194946 plgrid-te     wrap plggarst CG       4:34      2 p[0097,0100]'